In [1]:
from sklearn import metrics
import numpy as np
import torch
import argparse
import os

from matplotlib import pyplot as plt

import change_detection_pytorch as cdp
from change_detection_pytorch.datasets import LEVIR_CD_Dataset
from torch.utils.data import DataLoader

import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

from change_detection_pytorch.datasets import ChangeDetectionDataModule
from argparse import ArgumentParser
from tqdm import tqdm

cuda:0


In [2]:
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR']="localhost"
os.environ['MASTER_PORT']="12346"

dist.init_process_group(backend='nccl', init_method='env://')


In [3]:
from dataclasses import dataclass

@dataclass
class Args:
    experiment_name: str = 'tmp'
    backbone: str = 'Swin-B'
    encoder_weights: str = 'geopile'
    encoder_depth: int = 12
    dataset_name: str = 'OSCD'
    dataset_path: str = '/mnt/sxtn/aerial/change/OSCD/'
    fusion: str = 'diff'
    scale: str = None
    tile_size: int = 192
    mode: str = 'vanilla'
    batch_size: int = 116 // 4
    load_decoder = False

In [4]:
results = {}

In [5]:
scales = ['1x', '2x', '4x', '8x']

In [6]:
checkpoints = [
    '/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_overlap_seg_head_e150_lr3e-5/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/cdd_ibot_overlap_seg_head_e150_lr3e-5/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/cdd_ibot_overlap_seg_head/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_overlap_seg_head/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/cdd_ibot_overlap_seg_head_e150/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_overlap_seg_head_lr3e-5/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_overlap_seg_head_e150/best_model.pth',
    # '/auto/home/ani/change_detection.pytorch/checkpoints/cdd_ibot_overlap_seg_head_lr3e-5/best_model.pth',
]

In [7]:
def f1_bitwise(y_true, y_pred):
    TP = np.bitwise_and(y_true, y_pred).sum()
    FP = np.bitwise_and(y_pred, np.logical_not(y_true)).sum()
    FN = np.bitwise_and(np.logical_not(y_pred), y_true).sum()

    precision = TP / (TP + FP + 1e-10)
    recall = TP / (TP + FN + 1e-10)
    F1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    return F1

In [8]:
print(checkpoints)

['/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth']


In [9]:
from change_detection_pytorch.base.modules import Activation
from change_detection_pytorch.utils import base
from change_detection_pytorch.utils import functional as F

class CustomMetric(base.Metric):
            __name__ = 'custom'
        
            def __init__(self, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
                super().__init__(**kwargs)
                self.eps = eps
                self.threshold = threshold
                self.activation = Activation(activation)
                self.ignore_channels = ignore_channels
        
            def forward(self, y_pr, y_gt):
                y_pr = self.activation(y_pr)
                data['p'] = np.concatenate([data['p'], y_pr.cpu().numpy().astype('uint8')])
                data['t'] = np.concatenate([data['t'], y_gt.cpu().numpy().astype('uint8')])
                
                fscores = torch.tensor([F.f_score(p, g) for p, g in zip(y_pr, y_gt)])
                # plt.figure(figsize=(4,2))
                # plt.imshow((y_pr[0]*2+y_gt[0]).cpu().numpy(), cmap='nipy_spectral', vmax=4)
                # plt.title(f"F-score={fscores[0]:.3f}")
                # print("\n", y_pr.shape, y_gt.shape)
                # print((y_pr*y_gt).sum(), y_pr.sum(), y_gt.sum())
                return fscores.mean()

In [10]:
for checkpoint_path in checkpoints:
    results[checkpoint_path] = {}
    args = Args()  # Create an instance with default values

    if 'ibot' in checkpoint_path:
        if 'overlap' in checkpoint_path:
            args.backbone = 'overlap_ibot-B'
            args.encoder_weights = 'million_aid_overlap'
            args.encoder_depth = 12
            args.fusion = 'concat' 
            args.load_decoder = True 

        else:
            args.backbone = 'ibot-B'
            args.encoder_weights = 'million_aid'
            args.encoder_depth = 12
            
    elif 'satlas' in checkpoint_path:
        args.backbone = 'Swin-B'
        args.encoder_weights = 'satlas'
        args.encoder_depth = 12
    elif 'cmid' in checkpoint_path:
        args.backbone = 'Swin-B'
        args.encoder_weights = 'cmid'
        args.encoder_depth = 12
    
    if 'cdd' in checkpoint_path:
        args.dataset_name = 'CDD'
        args.dataset_path = '/mnt/sxtn/aerial/change/CDD/Real/subset/'
        args.batch_size = 32
        args.tile_size = 256 # it doesn't use
    elif 'levir' in checkpoint_path:
        args.dataset_name = 'LEVIR_CD'
        args.dataset_path = '/nfs/ap/mnt/sxtn/aerial/change/LEVIR_CD'
        args.batch_size = 64
        args.tile_size = 256 # it doesn't use
        
    print(args)
                            
    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    
    # args.fusion = 'abs_diff'
    
    model = cdp.UPerNet(
        encoder_depth=args.encoder_depth,
        encoder_name=args.backbone, # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights=args.encoder_weights, # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3, # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2, # model output channels (number of classes in your datasets)
        siam_encoder=True, # whether to use a siamese encoder
        fusion_form=args.fusion, # the form of fusing features from two branches. e.g. concat, sum, diff, or abs_diff.
        pretrained = args.load_decoder
    )
    model.to('cuda:{}'.format(dist.get_rank()))
    model = DDP(model)
    
    checkpoint = torch.load(checkpoint_path, map_location='cuda:{}'.format(dist.get_rank()))
    
    model.load_state_dict(checkpoint.state_dict())

    for scale in scales:
        if scale != '1x':
            args.scale = scale
            args.mode = 'wo_train_aug'
        if 'cdd' in args.dataset_name.lower():
            print('CDD', args.dataset_name)
            valid_dataset = LEVIR_CD_Dataset(f'{args.dataset_path}/test',
                                            sub_dir_1='A',
                                            sub_dir_2='B' if scale == '1x' else f'B_{scale}',
                                            img_suffix='.jpg',
                                            ann_dir=f'{args.dataset_path}/test/OUT',
                                            debug=False,
                                            seg_map_suffix='.jpg')
            valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)
            data = {
                'p': np.empty((0, 256, 256), dtype='uint8'),
                't': np.empty((0, 256, 256), dtype='uint8'),
                'f': []
            }


        elif 'levir' in args.dataset_name.lower():
            print('LEVIR', args.dataset_name, {args.dataset_path})
            valid_dataset = LEVIR_CD_Dataset(f'{args.dataset_path}/test',
                                            sub_dir_1='A_cut',
                                            sub_dir_2='B_cut' if scale == '1x' else f'B_cut_{scale}',
                                            img_suffix='.png',
                                            # ann_dir=f'{args.dataset_path}/test/OUT',
                                            ann_dir=f'{args.dataset_path}/test/OUT_cut',
                                            debug=False,
                                            test_mode=True,
                                            seg_map_suffix='.png')
            valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)
            data = {
                'p': np.empty((0, 256, 256), dtype='uint8'),
                't': np.empty((0, 256, 256), dtype='uint8'),
                'f': []
            }


        elif 'oscd' in args.dataset_name.lower():
            print('oscd', args.dataset_name)
            datamodule = ChangeDetectionDataModule(args.dataset_path, bands = ['B04', 'B02', 'B05'], patch_size=args.tile_size, mode=args.mode, scale=args.scale, batch_size=args.batch_size)
            datamodule.setup()
            
            valid_loader = datamodule.val_dataloader()
            
            data = {
                'p': np.empty((0, 192, 192), dtype='uint8'),
                't': np.empty((0, 192, 192), dtype='uint8'),
                'f': []
            }
            
        loss = cdp.utils.losses.CrossEntropyLoss()
        
        our_metrics = [
            cdp.utils.metrics.Fscore(activation='argmax2d'),
            cdp.utils.metrics.Precision(activation='argmax2d'),
            cdp.utils.metrics.Recall(activation='argmax2d'),
            CustomMetric(activation='argmax2d'),
        ]
        
        valid_epoch = cdp.utils.train.ValidEpoch(
            model,
            loss=loss,
            metrics=our_metrics,
            device=DEVICE,
            verbose=True,
        )
        
        valid_logs = valid_epoch.run(valid_loader)
                
        if 'cdd' in args.dataset_name.lower() or 'levir' in args.dataset_name.lower():
            fscores = []
            maps_t = []
            maps_p = []
            for p, t in tqdm(zip(data['p'], data['t'])):
                if p.sum() + t.sum() == 0:
                    fscores.append(0)
                else:
                    f1_real = metrics.f1_score(t.flatten(), p.flatten())
                    # f1_ours = f1_bitwise(t.flatten(), p.flatten())
                    fscores.append(
                        f1_real
                    )
                maps_t.append(t)
                maps_p.append(p)
            macro_f1 = np.mean(fscores)
            maps_t = np.vstack(maps_t)
            maps_p = np.vstack(maps_p)
            print(maps_t.shape, maps_p.shape)

            print(maps_t)
            print(maps_t.dtype)
            micro_f1 = f1_bitwise(maps_t, maps_p)
            maps = {'t':maps_t, 'p':maps_p}
        else:
            data['f'] = [y for x in valid_logs['filenames'] for y in x]
            cities = []
            coords = []
            for name in data['f']:
                name = name.split('/')[-1]
                _parts = name.split('_')
                city = '_'.join(_parts[:-1])
                coord = [int(t) for t in _parts[-1][1:-1].split(', ')]
                cities.append(city)
                coords.append(coord)
            unique_cities = set(cities)
            maps = {city: {
                't': np.zeros((1000, 1000)),
                'p': np.zeros((1000, 1000)),
            } for city in unique_cities}
            for city, coord, p, t in zip(cities, coords, data['p'], data['t']):
                x1,y1,x2,y2 = coord
                maps[city]['t'][y1:y2,x1:x2] = t
                maps[city]['p'][y1:y2,x1:x2] = p
            for city in tqdm(maps.keys()):
                maps[city]['fscore'] = metrics.f1_score(maps[city]['t'].flatten(), maps[city]['p'].flatten())
            micro_f1 = metrics.f1_score(
                np.concatenate([maps[city]['t'].flatten() for city in maps]),
                np.concatenate([maps[city]['p'].flatten() for city in maps]), 
            )
            macro_f1 = np.mean([maps[city]['fscore'] for city in maps]) 
            
        print(checkpoint_path, scale)
        print(macro_f1, micro_f1)
    
        results[checkpoint_path][scale] = {
            'maps': maps,
            'micro_f1': micro_f1,
            'macro_f1': macro_f1
        }

Args(experiment_name='tmp', backbone='ibot-B', encoder_weights='million_aid', encoder_depth=12, dataset_name='LEVIR_CD', dataset_path='/nfs/ap/mnt/sxtn/aerial/change/LEVIR_CD', fusion='diff', scale=None, tile_size=256, mode='vanilla', batch_size=64)
Pretrained weights found at /nfs/go/mnt/bolbol/alla/checkpoints/ibot_checkpoints/vitb_16/checkpoint0080.pth and loaded with msg: _IncompatibleKeys(missing_keys=['neck.lateral_convs.0.conv.weight', 'neck.lateral_convs.0.conv.bias', 'neck.lateral_convs.1.conv.weight', 'neck.lateral_convs.1.conv.bias', 'neck.lateral_convs.2.conv.weight', 'neck.lateral_convs.2.conv.bias', 'neck.lateral_convs.3.conv.weight', 'neck.lateral_convs.3.conv.bias', 'neck.convs.0.conv.weight', 'neck.convs.0.conv.bias', 'neck.convs.1.conv.weight', 'neck.convs.1.conv.bias', 'neck.convs.2.conv.weight', 'neck.convs.2.conv.bias', 'neck.convs.3.conv.weight', 'neck.convs.3.conv.bias'], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bi

2048it [00:09, 221.11it/s]


(524288, 256) (524288, 256)
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
uint8
/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth 1x
0.35452911298449163 0.8977437375153011
LEVIR LEVIR_CD {'/nfs/ap/mnt/sxtn/aerial/change/LEVIR_CD'}
Loaded 2048 images
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:59<00:00,  1.85s/it]


2048it [00:09, 222.78it/s]


(524288, 256) (524288, 256)
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
uint8
/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth 2x
0.3537505679389352 0.8971167305964438
LEVIR LEVIR_CD {'/nfs/ap/mnt/sxtn/aerial/change/LEVIR_CD'}
Loaded 2048 images
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:58<00:00,  1.83s/it]


2048it [00:09, 224.00it/s]


(524288, 256) (524288, 256)
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
uint8
/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth 4x
0.3515977558844875 0.8923869670706764
LEVIR LEVIR_CD {'/nfs/ap/mnt/sxtn/aerial/change/LEVIR_CD'}
Loaded 2048 images
valid: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:49<00:00,  1.54s/it]


2048it [00:09, 225.27it/s]


(524288, 256) (524288, 256)
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
uint8
/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth 8x
0.3418928398340948 0.8748598267186741


In [11]:
for checkpoint in checkpoints:
    print(checkpoint)
    for scale in scales:
        print(f"{scale} macro F1 = {results[checkpoint][scale]['macro_f1']:.3f}  micro-F1 = {results[checkpoint][scale]['micro_f1']:.3f}")
    print(' ')

/auto/home/ani/change_detection.pytorch/checkpoints/levir_ibot_scale_aug/best_model.pth
1x macro F1 = 0.355  micro-F1 = 0.898
2x macro F1 = 0.354  micro-F1 = 0.897
4x macro F1 = 0.352  micro-F1 = 0.892
8x macro F1 = 0.342  micro-F1 = 0.875
 


In [ ]:
for checkpoint in checkpoints:
    print(checkpoint)
    for scale in ['8x', '4x', '2x', '1x']:
        print(f",{results[checkpoint][scale]['micro_f1']:.3f}", end='')
    print(' ')

In [1]:
def ret_sqrt (x):
    return x*x

In [2]:
a = ret_sqrt(5), ret_sqrt(5)

In [4]:
c = ret_sqrt(5)
b= ret_sqrt(5)

In [7]:
a = c, b 

In [8]:
a

(25, 25)